In [ ]:
import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns

Using TensorFlow backend.


In [ ]:
df = pd.read_pickle('combined v1.pkl')

In [ ]:
features = ['next_order', 'next_park_factor_SLG', 'next_park_factor_OBP', 'S_avg', 'SB_avg', 
                    'BB_avg', 'HR_avg', 'CS_avg', 'pts_1000', 'ratio_100', 'ratio_250', 
                    'ratio_500','park_ratio_SLG','park_ratio_OBP', 'pts_500_p','whip_SO_p'
                    ,'ratio_2000','park_ratio_SLG_p','park_ratio_OBP_p']

#from Adrienne
features = ['next_order', 'park_ratio_SLG', 'next_park_factor_OBP', 'S_avg', 'SB_avg', 
                    'BB_avg', 'HR_avg', 'CS_avg', 'pts_1000_parkadj', 'ratio_100', 'ratio_250', 'ratio_500']

In [ ]:
# Split data
X_train = df[features][(df['year'] < 2017) & (df['year'] >= 2008)]
X_test = df[features][df['year'] >= 2017]

# Define y_train and y_test
y_train = df[['pts']][(df['year'] < 2017) & (df['year'] >= 2008)]
y_test = df[['pts']][df['year'] >= 2017]
print(X_train.shape)
print(X_test.shape)

(369867, 12)
(123130, 12)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model 1 with two hidden layers

In [ ]:
# Define model
model1 = Sequential()
model1.add(Dense(32, activation = 'relu'))
model1.add(Dense(32, activation = 'relu'))
model1.add(Dense(1))

In [ ]:
model1.compile(optimizer = 'adam', loss = 'mse', metrics = [keras.metrics.MeanAbsoluteError()])

In [ ]:
model1.fit(X_train_scaled, y_train['pts'].values, batch_size = 50, epochs = 10, verbose = 1, validation_split = 0.25)


Train on 277400 samples, validate on 92467 samples
Epoch 1/10
277400/277400 [==============================] - 19s 67us/step - loss: 87.1150 - mean_absolute_error: 7.1764 - val_loss: 87.5091 - val_mean_absolute_error: 7.1193
Epoch 2/10
277400/277400 [==============================] - 20s 72us/step - loss: 85.5207 - mean_absolute_error: 7.1211 - val_loss: 87.3856 - val_mean_absolute_error: 7.1121
Epoch 3/10
277400/277400 [==============================] - 19s 68us/step - loss: 85.4227 - mean_absolute_error: 7.1109 - val_loss: 87.4874 - val_mean_absolute_error: 7.1110
Epoch 4/10
277400/277400 [==============================] - 19s 68us/step - loss: 85.3721 - mean_absolute_error: 7.1151 - val_loss: 87.3571 - val_mean_absolute_error: 7.1094
Epoch 5/10
277400/277400 [==============================] - 19s 68us/step - loss: 85.3458 - mean_absolute_error: 7.1087 - val_loss: 87.3327 - val_mean_absolute_error: 7.1070
Epoch 6/10
277400/277400 [==============================] - 19s 69us/step - lo

In [ ]:
y_test['pred'] = model1.predict(X_test_scaled)
y_test['year'] = df[['year']][df['year'] >= 2017]

In [ ]:
# Calculate residuals, MAE, and RMSE 
y_test['residuals'] = y_test['pts'] - y_test['pred']
y_test['MAE'] = y_test['residuals'].abs()
y_test['RMSE'] = y_test['residuals']**2

# Calculate correlation, group it by year
cor = y_test[['pts','pred','year']].groupby('year').corr().reset_index()
cor = cor[cor['level_1'] == 'pts'][['year','pred']]

# Add year column
cor.set_index('year', inplace = True)

# Add correlation
cor.columns = ['correlation']

# Put all metrics together per year and total
cor.loc['Total','correlation'] =  y_test[['pts','pred']].corr().iloc[0, 1]
err = y_test[['year','MAE','RMSE']].groupby('year').mean()
err['RMSE'] = err['RMSE']**0.5
err.loc['Total', 'MAE'] = y_test['residuals'].abs().mean()
err.loc['Total', 'RMSE'] = y_test['RMSE'].mean()**0.5
cor = cor.join(err)

# Print result
print(cor)



       correlation       MAE       RMSE
year                                   
2017      0.160540  7.550233   9.889103
2018      0.174504  7.345305   9.517581
2019      0.180540  7.719740  10.083455
Total     0.172141  7.537815   9.831999


In [ ]:
y_train['pred'] = model1.predict(X_train_scaled)
y_train['year'] = df[['year']][(df['year'] < 2017) & (df['year'] >= 2008)]

In [ ]:
# Calculate residuals, MAE, and RMSE 
y_train['residuals'] = y_train['pts'] - y_train['pred']
y_train['MAE'] = y_train['residuals'].abs()
y_train['RMSE'] = y_train['residuals']**2

# Calculate correlation, group it by year
cor = y_train[['pts','pred','year']].groupby('year').corr().reset_index()
cor = cor[cor['level_1'] == 'pts'][['year','pred']]

# Add year column
cor.set_index('year', inplace = True)

# Add correlation
cor.columns = ['correlation']

# Put all metrics together per year and total
cor.loc['Total','correlation'] =  y_train[['pts','pred']].corr().iloc[0, 1]
err = y_train[['year','MAE','RMSE']].groupby('year').mean()
err['RMSE'] = err['RMSE']**0.5
err.loc['Total', 'MAE'] = y_train['residuals'].abs().mean()
err.loc['Total', 'RMSE'] = y_train['RMSE'].mean()**0.5
cor = cor.join(err)

# Print result
print(cor)



       correlation       MAE      RMSE
year                                  
2008      0.181743  7.232444  9.469843
2009      0.176168  7.267132  9.494283
2010      0.181575  7.097225  9.249201
2011      0.186697  7.039943  9.116126
2012      0.185339  7.140514  9.268734
2013      0.170293  7.015724  9.080985
2014      0.176699  6.891841  8.852921
2015      0.181868  7.069267  9.201252
2016      0.177069  7.322750  9.555173
Total     0.181265  7.119556  9.256564


# Model 2 with three hidden layers

In [ ]:
# Define model
model11 = Sequential()
model11.add(Dense(64, activation = 'relu'))
model11.add(Dense(64, activation = 'relu'))
model11.add(Dense(64, activation = 'relu'))
model11.add(Dense(1))

In [ ]:
model11.compile(optimizer = 'adam', loss = 'mse', metrics = [keras.metrics.MeanAbsoluteError()])

In [ ]:
history11 = model11.fit(X_train_scaled, y_train['pts'].values, batch_size = 50, epochs = 15, verbose = 1, validation_split = 0.25)

Epoch 1/15
5548/5548 [==============================] - 11s 2ms/step - loss: 86.6562 - mean_absolute_error: 7.1060 - val_loss: 87.8494 - val_mean_absolute_error: 7.0401
Epoch 2/15
5548/5548 [==============================] - 11s 2ms/step - loss: 85.5630 - mean_absolute_error: 7.0872 - val_loss: 87.7409 - val_mean_absolute_error: 7.3186
Epoch 3/15
5548/5548 [==============================] - 11s 2ms/step - loss: 85.4512 - mean_absolute_error: 7.0849 - val_loss: 87.4588 - val_mean_absolute_error: 7.1262
Epoch 4/15
5548/5548 [==============================] - 11s 2ms/step - loss: 85.3903 - mean_absolute_error: 7.0839 - val_loss: 87.3250 - val_mean_absolute_error: 7.1956
Epoch 5/15
5548/5548 [==============================] - 11s 2ms/step - loss: 85.3363 - mean_absolute_error: 7.0831 - val_loss: 87.4300 - val_mean_absolute_error: 7.0961
Epoch 6/15
5548/5548 [==============================] - 11s 2ms/step - loss: 85.3145 - mean_absolute_error: 7.0833 - val_loss: 87.4080 - val_mean_absolute_

In [ ]:
y_test['pred'] = model11.predict(X_test_scaled)
y_test['year'] = df_combined[['year']][df_combined['year'] >= 2017]

In [ ]:
# Calculate residuals, MAE, and RMSE 
y_test['residuals'] = y_test['pts'] - y_test['pred']
y_test['MAE'] = y_test['residuals'].abs()
y_test['RMSE'] = y_test['residuals']**2

# Calculate correlation, group it by year
cor = y_test[['pts','pred','year']].groupby('year').corr().reset_index()
cor = cor[cor['level_1'] == 'pts'][['year','pred']]

# Add year column
cor.set_index('year', inplace = True)

# Add correlation
cor.columns = ['correlation']

# Put all metrics together per year and total
cor.loc['Total','correlation'] =  y_test[['pts','pred']].corr().iloc[0, 1]
err = y_test[['year','MAE','RMSE']].groupby('year').mean()
err['RMSE'] = err['RMSE']**0.5
err.loc['Total', 'MAE'] = y_test['residuals'].abs().mean()
err.loc['Total', 'RMSE'] = y_test['RMSE'].mean()**0.5
cor = cor.join(err)

# Print result
print(cor)

       correlation       MAE       RMSE
year                                   
2017      0.157453  7.636833   9.891131
2018      0.169277  7.425385   9.535670
2019      0.177412  7.788645  10.082432
Total     0.168561  7.616377   9.838193


In [ ]:
y_train['pred'] = model11.predict(X_train_scaled)
y_train['year'] = df[['year']][(df['year'] < 2017) & (df['year'] >= 2008)]

In [ ]:
# Calculate residuals, MAE, and RMSE 
y_train['residuals'] = y_train['pts'] - y_train['pred']
y_train['MAE'] = y_train['residuals'].abs()
y_train['RMSE'] = y_train['residuals']**2

# Calculate correlation, group it by year
cor = y_train[['pts','pred','year']].groupby('year').corr().reset_index()
cor = cor[cor['level_1'] == 'pts'][['year','pred']]

# Add year column
cor.set_index('year', inplace = True)

# Add correlation
cor.columns = ['correlation']

# Put all metrics together per year and total
cor.loc['Total','correlation'] =  y_train[['pts','pred']].corr().iloc[0, 1]
err = y_train[['year','MAE','RMSE']].groupby('year').mean()
err['RMSE'] = err['RMSE']**0.5
err.loc['Total', 'MAE'] = y_train['residuals'].abs().mean()
err.loc['Total', 'RMSE'] = y_train['RMSE'].mean()**0.5
cor = cor.join(err)

# Print result
print(cor)

       correlation       MAE      RMSE
year                                  
2008      0.196894  7.138109  9.458456
2009      0.195673  7.171892  9.473373
2010      0.198455  6.981233  9.221441
2011      0.200163  6.930880  9.093101
2012      0.198935  7.035025  9.247349
2013      0.189321  6.890180  9.043918
2014      0.190894  6.764901  8.816184
2015      0.197755  6.955191  9.175720
2016      0.191346  7.224553  9.540036
Total     0.197190  7.010111  9.232400
